# **21AIE315 AI IN SPEECH PROCESSING**

**Automatic Speech Recognition (ASR) for Hindi using XLSR-Wav2Vec2**

    Deepthi Sudharsan |  CB.EN.U4AIE19022
    Harish K          |  CB.EN.U4AIE19029
    U Asmitha         |  CB.EN.U4AIE19065
    Roshan Tushar S   |  CB.EN.U4AIE19071

### Installing the necessary packages

In [83]:
%%capture
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torchaudio
!pip install librosa
!pip install jiwer

## Prepare Data, Tokenizer, Feature Extractor

### XLSR-Wav2Vec2
 Wav2Vec2CTCTokenizer --> Tokenizer
 Wav2Vec2FeatureExtractor --> Feature Extractor

### Create Wav2Vec2CTCTokenizer

In [84]:
# Downloading the Dataset
!wget https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-6.1-2020-12-11/hi.tar.gz

--2022-05-09 20:30:14--  https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-6.1-2020-12-11/hi.tar.gz
Resolving voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com (voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com)... 52.92.210.249
Connecting to voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com (voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com)|52.92.210.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21424045 (20M) [application/octet-stream]
Saving to: ‘hi.tar.gz.1’

hi.tar.gz.1         100%[===================>]  20.43M  11.9MB/s    in 1.7s    

2022-05-09 20:30:17 (11.9 MB/s) - ‘hi.tar.gz.1’ saved [21424045/21424045]



In [85]:
# Unzipping the dataset
!tar -xvzf hi.tar.gz

cv-corpus-6.1-2020-12-11/hi/
cv-corpus-6.1-2020-12-11/hi/clips/
cv-corpus-6.1-2020-12-11/hi/dev.tsv
cv-corpus-6.1-2020-12-11/hi/invalidated.tsv
cv-corpus-6.1-2020-12-11/hi/other.tsv
cv-corpus-6.1-2020-12-11/hi/reported.tsv
cv-corpus-6.1-2020-12-11/hi/test.tsv
cv-corpus-6.1-2020-12-11/hi/train.tsv
cv-corpus-6.1-2020-12-11/hi/validated.tsv
cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23795238.mp3
cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23795239.mp3
cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23795240.mp3
cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23795241.mp3
cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23795242.mp3
cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23795243.mp3
cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23795244.mp3
cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23795245.mp3
cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23795246.mp3
cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23795247.mp3
cv-corpus-6.1-2020-12-11/hi/cl

The data is now saved under `./cv-corpus-6.1-2020-12-11`. We can leverage datasets now to preprocess the dataset.

In [86]:
from datasets import load_dataset, load_metric
# We merge the train and validation as the dataset is very small
common_voice_train = load_dataset("common_voice", "hi", data_dir="./cv-corpus-6.1-2020-12-11", split="train+validation")
common_voice_test = load_dataset("common_voice", "hi", data_dir="./cv-corpus-6.1-2020-12-11", split="test")

Using custom data configuration hi-24242e0bc11eddb3
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e)
Using custom data configuration hi-24242e0bc11eddb3
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e)


### Distribution of files between train and test

In [87]:
print('Training Data :')
print(common_voice_train)
print('\n')
print('Testing Data :')
print(common_voice_test)

Training Data :
Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 292
})


Testing Data :
Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 127
})


##### Removing Unnecessary Features from the dataset

In [88]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [89]:
print('Training Data modified :')
print(common_voice_train)
print('\n')
print('Testing Data modified :')
print(common_voice_test)

Training Data modified :
Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 292
})


Testing Data modified :
Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 127
})


##### Function to Randomly print samples

In [90]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [91]:
show_random_elements(common_voice_train.remove_columns(["path"]))

,audio,sentence
0,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c96485295dfcd0d08e461a9bab70c7431c5b032810e282334a76757b3eb2d753/cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23795252.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",उसने मेरे खिलाफ़ हथियार उठाया।
1,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c96485295dfcd0d08e461a9bab70c7431c5b032810e282334a76757b3eb2d753/cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23852779.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",उसे फ़्रानसीसी जर्मन से ज़्यादा अच्छी लगती है।
2,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c96485295dfcd0d08e461a9bab70c7431c5b032810e282334a76757b3eb2d753/cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23852764.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",सिद्धार्थ मेरा अच्छा दोस्त है और हमेशा रहेगा: आलिया
3,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c96485295dfcd0d08e461a9bab70c7431c5b032810e282334a76757b3eb2d753/cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23852752.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",कितने सेब हैं?
4,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c96485295dfcd0d08e461a9bab70c7431c5b032810e282334a76757b3eb2d753/cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23849294.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",टीवी चालू कीजिए।
5,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c96485295dfcd0d08e461a9bab70c7431c5b032810e282334a76757b3eb2d753/

##### Removing the special characters

In addition, we normalize the text to only have lower case letters and append a word separator token at the end.

In [92]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [93]:
common_voice_train = common_voice_train.map(remove_special_characters, remove_columns=["sentence"])
common_voice_test = common_voice_test.map(remove_special_characters, remove_columns=["sentence"])

Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-0a7eaa2ceef1d3f2.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-75b8f36eb0c0749b.arrow


In [94]:
show_random_elements(common_voice_train.remove_columns(["path"]),num_examples = 5)

,audio,text
0,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c96485295dfcd0d08e461a9bab70c7431c5b032810e282334a76757b3eb2d753/cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23830501.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",सिंगापुर को मलाय में सिंगापुरा कहते हैं।
1,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c96485295dfcd0d08e461a9bab70c7431c5b032810e282334a76757b3eb2d753/cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23832737.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",मैं तुमसे नहीं बोलती
2,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c96485295dfcd0d08e461a9bab70c7431c5b032810e282334a76757b3eb2d753/cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23849378.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",गृह राज्य मंत्री बोले जामिया हिंसा के पीछे विदेशी ताकतों का साथ
3,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c96485295dfcd0d08e461a9bab70c7431c5b032810e282334a76757b3eb2d753/cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23852758.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",मैं तुम्हें एक तोहफ़ा दूँगा।
4,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c96485295dfcd0d08e461a9bab70c7431c5b032810e282334a76757b3eb2d753/cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23964485.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",यह लिफ़्ट छटी मंज़िल तक ही जाती है।


##### Extracting the grammer for our model

In [95]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return { "all_text": [all_text],"vocab": [vocab]}

In [96]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Union of all sets of vocab

In [97]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [98]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 17,
 "'": 3,
 'a': 35,
 'e': 44,
 'f': 31,
 'i': 42,
 'l': 14,
 'm': 62,
 'p': 11,
 'r': 61,
 'u': 58,
 'w': 22,
 'ँ': 13,
 'ं': 21,
 'ः': 67,
 'अ': 56,
 'आ': 50,
 'इ': 20,
 'ई': 52,
 'उ': 49,
 'ऊ': 47,
 'ए': 40,
 'ऐ': 19,
 'ऑ': 6,
 'ओ': 53,
 'औ': 2,
 'क': 39,
 'ख': 69,
 'ग': 16,
 'घ': 55,
 'च': 23,
 'छ': 46,
 'ज': 1,
 'झ': 5,
 'ट': 0,
 'ठ': 70,
 'ड': 54,
 'ढ': 8,
 'ण': 51,
 'त': 34,
 'थ': 30,
 'द': 45,
 'ध': 68,
 'न': 60,
 'प': 12,
 'फ': 41,
 'ब': 18,
 'भ': 7,
 'म': 59,
 'य': 24,
 'र': 28,
 'ल': 15,
 'व': 63,
 'श': 48,
 'ष': 4,
 'स': 26,
 'ह': 43,
 '़': 33,
 'ा': 38,
 'ि': 9,
 'ी': 10,
 'ु': 66,
 'ू': 32,
 'ृ': 65,
 'े': 27,
 'ै': 36,
 'ॉ': 25,
 'ो': 71,
 'ौ': 64,
 '्': 37,
 'ड़': 57,
 '।': 29}

In [99]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [100]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)
# length of the Vocabulary

74

Now our vocabulary is complete and consists of 74 tokens, which means that the linear layer that we will add on top of the pretrained XLSR-Wav2Vec2 checkpoint will have an output dimension of 74.

saving the vocabulary as a json file.

In [101]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In a final step, we use the json file to instantiate an object of the `Wav2Vec2CTCTokenizer` class.

In [102]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

### Create XLSR-Wav2Vec2 Feature Extractor

Speech is a continous signal and we need to sample it.

Sampling Rate : 16kHz
Downsampling our fine-tuning data to 16kHz

A XLSR-Wav2Vec2 feature extractor object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal ${}^2$.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be *zero-mean-unit-variance* normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an `attention_mask` for batched inference. In general, XLSR-Wav2Vec2 models should **always** make use of the `attention_mask`.

In [103]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

The feature extractor and tokenizer are *wrapped* into a single `Wav2Vec2Processor` class so that one only needs a `model` and `processor` object.

In [104]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [105]:
#Mounting the drive to save the model

In [106]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [107]:
 processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-hindi-demo")

Feature extractor saved in /content/gdrive/MyDrive/wav2vec2-large-xlsr-hindi-demo/preprocessor_config.json
tokenizer config file saved in /content/gdrive/MyDrive/wav2vec2-large-xlsr-hindi-demo/tokenizer_config.json
Special tokens file saved in /content/gdrive/MyDrive/wav2vec2-large-xlsr-hindi-demo/special_tokens_map.json


Next, we can prepare the dataset.

### Preprocess Data

Converting the audio files to 1-d vectors.


In [108]:
common_voice_train[0]

{'audio': {'array': array([0.        , 0.        , 0.        , ..., 0.01033163, 0.00594568,
         0.00187945], dtype=float32),
  'path': '/root/.cache/huggingface/datasets/downloads/extracted/c96485295dfcd0d08e461a9bab70c7431c5b032810e282334a76757b3eb2d753/cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23795238.mp3',
  'sampling_rate': 48000},
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/c96485295dfcd0d08e461a9bab70c7431c5b032810e282334a76757b3eb2d753/cv-corpus-6.1-2020-12-11/hi/clips/common_voice_hi_23795238.mp3',
 'text': 'तुम्हारे पास तीन महीने बचे हैं। '}

File Format used is `.mp3`.
[`torchaudio`](https://pytorch.org/audio/stable/index.html) library works best for reading in `.mp3` data. 


An audio file usually stores both its values and the sampling rate with which the speech signal was digitalized. We want to store both in the dataset and write a `map(...)` function accordingly.

In [109]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

In [110]:
common_voice_train = common_voice_train.map(speech_file_to_array_fn, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names)

Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-e071f56ecca4a183.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-68cceb303387937d.arrow


Common Voice is sampled at 48kHz, we need to resample the audio files to 16kHz. 

-->librosa library to downsample the data.

In [111]:
import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [112]:
common_voice_train = common_voice_train.map(resample, num_proc=4)
common_voice_test = common_voice_test.map(resample, num_proc=4)

Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-0dc882868869aefe.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-7740a3bdf8c3efca.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-33557b131b89cc50.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-9ad7a9179ba06095.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-1d8a1105c9207227.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-aa97831fd9bc3ad5.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-9f41e83ebe69da94.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-ae2312796762122a.arrow


In [113]:
##### Playing a random audio file


In [114]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train))

ipd.Audio(data=np.asarray(common_voice_train[rand_int]["speech"]), autoplay=True, rate=16000)

In [115]:
rand_int = random.randint(0, len(common_voice_train))

print("Target text:", common_voice_train[rand_int]["target_text"])
print("Input array shape:", np.asarray(common_voice_train[rand_int]["speech"]).shape)
print("Sampling rate:", common_voice_train[rand_int]["sampling_rate"])

Target text: सर्वदलीय प्रतिनिधिमंडल में शामिल प्रणव व चिदंबरम 
Input array shape: (97152,)
Sampling rate: 16000


In [116]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [117]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, batch_size=8, num_proc=4, batched=True)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, batch_size=8, num_proc=4, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-64ad55c1b8a38618.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-5e1457589e551eea.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-74be250a3a26d3bb.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-b067d664cf5c82c6.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-3d76f5149d8ee0b2.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-31ab86a7ae47344f.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-6c39e7c9b0cc222a.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-d812339feafef89d.arrow


## Training

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, XLSR-Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning XLSR-Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

In [118]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [119]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

The evaluation metric --> word error rate (WER)

In [120]:
wer_metric = load_metric("wer")

The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style ${}^1$.

In [121]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [122]:
from transformers import Wav2Vec2ForCTC
# defining the model
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

loading configuration file https://huggingface.co/facebook/wav2vec2-large-xlsr-53/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/8508c73cd595eb416a1d517b90762416c0bc6cfbef529578079aeae4d8c14336.7581ed2ee0c677f1e933180df51bd1a668c4a2b6d5fd1297d32069373dac097c
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "

The first component of XLSR-Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal.

This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore. 
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [123]:
model.freeze_feature_extractor()

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1677: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


In a final step, we define all parameters related to training. 
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Common Voice dataset and might be suboptimal for other speech datasets.



In [124]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-hindi-demo",
  # output_dir="./wav2vec2-large-xlsr-Tamil-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=2,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Now, all instances can be passed to Trainer and we are ready to start training!

In [125]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend




---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.

${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`.

### Training

In [126]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 292
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 18


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=18, training_loss=24.439626057942707, metrics={'train_runtime': 86.8653, 'train_samples_per_second': 6.723, 'train_steps_per_second': 0.207, 'total_flos': 9.689602882046976e+16, 'train_loss': 24.439626057942707, 'epoch': 1.95})

In [127]:
!pip install transformers

In [128]:
import torch
import torchaudio
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re

test_dataset = load_dataset("common_voice", "hi", split="test")
wer = load_metric("wer")

processor = Wav2Vec2Processor.from_pretrained("theainerd/Wav2Vec2-large-xlsr-hindi")
model = Wav2Vec2ForCTC.from_pretrained("theainerd/Wav2Vec2-large-xlsr-hindi")
model.to("cuda")

resampler = torchaudio.transforms.Resample(48_000, 16_000)

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“]'

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
  batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
  speech_array, sampling_rate = torchaudio.load(batch["path"])
  batch["speech"] = resampler(speech_array).squeeze().numpy()
  return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(batch):
  inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

  with torch.no_grad():
      logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits

      pred_ids = torch.argmax(logits, dim=-1)
      batch["pred_strings"] = processor.batch_decode(pred_ids)
      return batch

result = test_dataset.map(evaluate, batched=True, batch_size=8)

print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))

Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/hi/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e)
loading feature extractor configuration file https://huggingface.co/theainerd/Wav2Vec2-large-xlsr-hindi/resolve/main/preprocessor_config.json from cache at /root/.cache/huggingface/transformers/a064e188e889210c0701cc4883f9f0aac8985ce34dc37ab022f0ed1976b9f16b.fcd266b775b7f33ba9b607a0fee7cc615aeb2eb281586f046280492ea380ae23
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

loading configuration file https://huggingface.co/theainerd/Wav2Vec2-large-xlsr-hindi/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/7c07eaf6b65b79fff4688d42a0f765c386e1a696c1b6d3fd9ad40fdfed7f50a7.e480a0fd69da2949517209a96b1fe0697a34cd

  0%|          | 0/127 [00:00<?, ?ex/s]

  0%|          | 0/16 [00:00<?, ?ba/s]

WER: 73.201856


In [129]:
input_dict = processor(common_voice_test["input_values"][0], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [130]:
common_voice_test_transcription = load_dataset("common_voice", "hi", data_dir="./cv-corpus-6.1-2020-12-11", split="test")

Using custom data configuration hi-24242e0bc11eddb3
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/hi-24242e0bc11eddb3/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e)


Finally, we can decode the example.

In [131]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_transcription["sentence"][0].lower())

Prediction:
कानस्टुबल बड मनोछ बाज पे दिखा रहें है ताडब

Reference:
"कांस्टेबल बन मनोज बाजपेयी दिखा रहे हैं 'तांडव'"


# Using the model live to predict the speech

In [132]:
%%capture

!apt-get -y install sox ffmpeg
!pip install transformers ffmpeg-python sox

!wget https://raw.githubusercontent.com/harveenchadha/bol/main/demos/colab/record.py

In [133]:
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

def load_model():
    # load pretrained model
    processor = Wav2Vec2Processor.from_pretrained("theainerd/Wav2Vec2-large-xlsr-hindi")
    model = Wav2Vec2ForCTC.from_pretrained("theainerd/Wav2Vec2-large-xlsr-hindi")
    return processor, model

processor, model = load_model()

loading feature extractor configuration file https://huggingface.co/theainerd/Wav2Vec2-large-xlsr-hindi/resolve/main/preprocessor_config.json from cache at /root/.cache/huggingface/transformers/a064e188e889210c0701cc4883f9f0aac8985ce34dc37ab022f0ed1976b9f16b.fcd266b775b7f33ba9b607a0fee7cc615aeb2eb281586f046280492ea380ae23
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

loading configuration file https://huggingface.co/theainerd/Wav2Vec2-large-xlsr-hindi/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/7c07eaf6b65b79fff4688d42a0f765c386e1a696c1b6d3fd9ad40fdfed7f50a7.e480a0fd69da2949517209a96b1fe0697a34cd7bd1ca6e49485eceb77cc415fa
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing`

In [134]:
def parse_transcription(wav_file):
    # load audio    
    audio_input, sample_rate = sf.read(wav_file)

    # pad input values and return pt tensor
    input_values = processor(audio_input, sampling_rate=16_000, return_tensors="pt").input_values

    # INFERENCE
    # retrieve logits & take argmax
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    # transcribe
    transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)
    return transcription

## Record file using colab

In [135]:
from record import record_audio

record_audio('test')

output_file: test.wav already exists and will be overwritten on build


## Run Model on recorded file

In [136]:
parse_transcription('test.wav')

'तुम्हार नाम क्या है'